In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-04-22 14:07:35,926 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-22 14:07:35,928 - settings - Setting database.user to celiib
INFO - 2021-04-22 14:07:35,929 - settings - Setting database.password to newceliipass
INFO - 2021-04-22 14:07:35,946 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-22 14:07:35,947 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-22 14:07:35,960 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-22 14:07:36,335 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-22 14:07:36,377 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-22 14:07:36,378 - settings - Setting database.user to celiib
INFO - 2021-04-22 14:07:36,379 - settings - Setting database.password to newceliipass
INFO - 2021-04-22 14:07:36,380 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-22 14:07:36,381 - settings - Setting database.user to celiib
INFO - 2021-04-22 14:07:36,382 - settings - Setting database.password to newceliipass
INFO - 2021-04-22 14:07:36,384 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-22 14:07:36,782 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import system_utils as su

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-22 14:07:36,883 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-22 14:07:36,884 - settings - Setting database.user to celiib
INFO - 2021-04-22 14:07:36,885 - settings - Setting database.password to newceliipass
INFO - 2021-04-22 14:07:36,886 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-22 14:07:36,888 - settings - Setting database.user to celiib
INFO - 2021-04-22 14:07:36,889 - settings - Setting database.password to newceliipass
INFO - 2021-04-22 14:07:36,890 - settings - Setting enable_python_native_blobs to True


Sleeping 123 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-04-22 14:07:37,345 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 3356


In [8]:
# minnie.DecompositionSplit.drop()
# schema.external['decomposition'].delete(delete_external_files=True)

In [9]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & minnie.AutoProofreadValidationSegment()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135212632448,0,0.25,30.00,7,0,1,=BLOB=,627122,1290246,0,1,1,0,1,6,187,70,7319056.097799392,2276919.152335003,30129.669,99.269,99.269,439.937,418.696,1754,0.00023964838861221082,9.379679144385026,4385433.955,83,0.000399960418512334,21.133,413784359369.2133,139776566.6219406,56535.20807056324,94354.25629827169,4985353727.339919,1967.516
864691135589906955,0,0.25,30.00,7,0,1,=BLOB=,818368,1656604,0,1,1,0,1,8,218,111,8598179.206041662,3827546.03435727,22313.719,168.448,168.448,493.876,480.402,2140,0.0002488899043295459,9.81651376146789,5942399.424,121,0.000360123890588207,17.686,396220438465.8728,97877370.64833069,46081.900478122334,66676.8438462128,3274549078.230354,3708.4844
864691135748575017,0,0.25,30.00,7,0,1,=BLOB=,681153,1375265,0,1,1,0,1,5,112,53,5384926.828584723,2536763.607236986,25607.565,386.819,308.988,564.121,542.684,2515,0.0004670444149119473,22.455357142857142,4426430.2,82,0.0005681779416740831,30.671,680651467627.4121,157147420.03702384,126399.39024135306,153769.84090416972,8300627653.99283,3446.2602
864691135785277636,0,0.25,30.00,7,0,1,=BLOB=,910129,1842216,0,1,1,0,1,6,164,76,7465007.587399431,3090643.039568483,35514.775,371.67,295.793,549.873,514.541,3841,0.0005145339713362678,23.420731707317074,6195740.612,121,0.0006199420280056102,31.744,915521137387.6455,133366091.2866567,122641.6887951997,147766.21468214,7566290391.633434,4334.6605
864691135974454895,0,0.25,30.00,7,0,1,=BLOB=,673972,1339436,0,1,1,0,1,7,172,97,7874419.244368863,3916099.306968058,28135.979,258.047,239.96,529.967,510.996,1807,0.00022947724066028333,10.505813953488373,5309190.657,104,0.00034035319443982063,17.375,304869209834.06915,106299788.58333333,38716.40566408583,57422.91613357466,2931434709.9429727,2736.5303
864691136003930314,0,0.25,30.00,7,0,1,=BLOB=,567120,1129806,0,1,1,0,1,7,115,52,5706422.20255445,2274275.562747183,28837.481,302.662,274.596,534.692,5

In [15]:
import numpy as np
import time
import proofreading_utils as pru

split_version = 0
split_version = 1 #fixed the problem with split from suggestions
split_version = 2 #fixed the problem with split from suggestions

verbose = True

@schema
class DecompositionSplit(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index: tinyint unsigned  #the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    split_version: tinyint unsigned  #the version of the splitting algorithm used
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    n_splits             : int unsigned                 # the number of cuts required to help split the neuron
    split_success        : tinyint unsigned             # the successfulness of the splitting
    
    n_error_limbs_cancelled : tinyint unsigned     # number of limbs that couldn't be resolved and cancelled out        
    n_same_soma_limbs_cancelled : tinyint unsigned     # number of same soma touching limbs that couldn't be resolved and cancelled out
    n_multi_soma_limbs_cancelled : tinyint unsigned     # number of multi soma touching limbs that couldn't be resolved and cancelled out        
    
    error_imbs_cancelled_area=NULL : double            # the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas
    error_imbs_cancelled_skeletal_length = NULL : double #the total skeletal length (in microns) of the limbs that were called out because could not be resolved
    
    split_results: longblob #will store the results of how to split the limbs of neuron objects from original neuron
    decomposition: <decomposition>
    
    
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    max_soma_n_faces:  int unsigned                 # The largest number of faces of the somas
    max_soma_volume:  int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
    
    ''' Old keysource used for inhibitory excitatory check
    classified_table = (minnie.BaylorManualCellType() &
                        'nucleus_version=3' & 
                        "(cell_type = 'excitatory') or  (cell_type = 'inhibitory')")
    
    key_source = ((minnie.Decomposition & 
                (minnie.NeuronSplitSuggestions.proj()) & 
                (classified_table.proj()) 
                & f"n_somas<{max_n_somas}" & "n_error_limbs>0"))'''
    
    # This keysource acounts that you could have more than 1 possible soma but not a significant limb connecting them (no error limbs)
    key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & minnie.AutoProofreadValidationSegment()
    
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decomposed neurong object from Decomposition table
        3) Run the multi_soma split suggestions algorithm
        4) Get the number of splits required for this neuron
        5) Split the neuron into a list of neuron objects
        6) For each neuron object in the list:
        - get the number of errored limbs (to indicate the success type)
        - Change the description to include the multiplicity
        - Compute the information on the largest soma faces and volume
        - Save the neuron object to the external
        - Add the new write key to a list to commit 
        7) Write all of the keys 
        """
        
        whole_pass_time = time.time()
        
        
        
        # 1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        
        
        # 2) Get the decomposed neuron object from Decomposition table and the split suggestions
        neuron_obj = (minnie.Decomposition & key).fetch1("decomposition")
        
        """ Old way that downloaded from another table
        # 3) Retrieve the multi soma suggestions
        split_results = (minnie.NeuronSplitSuggestions & key).fetch1("split_results")
        """
        #3) Calculated the split results
        split_results = pru.multi_soma_split_suggestions(neuron_obj,plot_intermediates=False)
        
        # 4) Get the number of splits required for this neuron
        n_paths_cut = pru.get_n_paths_cut(split_results)

        if verbose:
            print(f"n_paths_cut = {n_paths_cut}")
            
            
        # 5) Split the neuron into a list of neuron objects
        (neuron_list,
        neuron_list_errored_limbs_area,
         neuron_list_errored_limbs_skeletal_length,
        neuron_list_n_multi_soma_errors,
        neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj,
                        limb_results=split_results,
                                       verbose=verbose,
                                        return_error_info=True
                                            )
        
        print(f"neuron_list = {neuron_list}")
        print(f"neuron_list_errored_limbs_area = {neuron_list_errored_limbs_area}")
        print(f"neuron_list_n_multi_soma_errors = {neuron_list_n_multi_soma_errors}")
        print(f"neuron_list_n_same_soma_errors = {neuron_list_n_same_soma_errors}")
        
        
        if verbose:
            print(f"Number of neurons: {len(neuron_list)}")
        
        neuron_entries = []
        for neuron_idx in range(len(neuron_list)):
            
            """
            # 6) For each neuron object in the list:
            # - get the number of errored limbs (to indicate the success type)
            # - Compute the information on the largest soma faces and volume
            # - Save the neuron object to the external
            # - Add the new write key to a list to commit 
            """
            n = neuron_list[neuron_idx]
            
            error_imbs_cancelled_area = neuron_list_errored_limbs_area[neuron_idx]
            error_imbs_cancelled_skeletal_length = neuron_list_errored_limbs_skeletal_length[neuron_idx]
            n_multi_soma_limbs_cancelled = neuron_list_n_multi_soma_errors[neuron_idx]
            n_same_soma_limbs_cancelled = neuron_list_n_same_soma_errors[neuron_idx]
            
            
            #for n in neuron_list:
            #     nviz.visualize_neuron(n,
            #                          limb_branch_dict="all")

            # - get the number of errored limbs (to indicate the success type)
            if n.n_error_limbs == 0:
                split_success = 0
            elif n.multi_soma_touching_limbs == 0:
                split_successs = 1
            elif n.same_soma_multi_touching_limbs == 0:
                split_success = 2
            else:
                split_success = 3

            if verbose:
                print(f"split_success = {split_success}")

            # - Compute the information on the largest soma faces and volume
            soma_volumes = [n[k].volume/1000000000 for k in n.get_soma_node_names()] 
            soma_n_faces = [len(n[k].mesh.faces) for k in n.get_soma_node_names()] 

            largest_n_faces = np.max(soma_n_faces)
            largest_volume = np.max(soma_volumes)

            if verbose:
                print(f"largest_n_faces = {largest_n_faces}")
                print(f"largest_volume = {largest_volume}")

            if "split" not in n.description:
                n.description += "_soma_0_split"
                
            #6) Save the file in a certain location
            if True:
                save_time = time.time()
                ret_file_path = n.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                                  return_file_path=True,
                                                 export_mesh=False,
                                                 suppress_output=True)

                ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
                print(f"Save time = {time.time() - save_time}")
            else:
                print("Storing a dummy value for neuron")
                ret_file_path_str = "dummy"



            #7) Pass stats and file location to insert
            new_key = dict(key,
                           split_index = neuron_idx,
                           split_version = split_version,
                           
                           multiplicity=len(neuron_list),

                           n_splits = n_paths_cut,
                           split_success = split_success,
                           
                           n_error_limbs_cancelled = len(error_imbs_cancelled_area),
                           
                           n_multi_soma_limbs_cancelled =n_multi_soma_limbs_cancelled,
                           n_same_soma_limbs_cancelled = n_same_soma_limbs_cancelled,
                           error_imbs_cancelled_area = np.round(np.sum(error_imbs_cancelled_area),4),
                           error_imbs_cancelled_skeletal_length = np.round(np.sum(error_imbs_cancelled_skeletal_length)/1000,4),
                           
                           split_results=split_results,

                           max_soma_n_faces = largest_n_faces,
                           max_soma_volume = largest_volume,


                           decomposition=ret_file_path_str,
                           n_vertices=len(n.mesh.vertices),
                           n_faces=len(n.mesh.faces),
                           run_time=np.round(time.time() - whole_pass_time,4)
                          )

            stats_dict = n.neuron_stats()
            new_key.update(stats_dict)
            
            keys_to_delete = ["axon_length",
            "axon_area",
                             "n_boutons"]

            for k_to_delete in keys_to_delete:
                del new_key[k_to_delete]


            neuron_entries.append(new_key)

        
        self.insert(neuron_entries, allow_direct_insert=True, skip_duplicates=True)
        

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - whole_pass_time} ------")

# Running the Populate

In [28]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
((curr_table) & "timestamp>'2021-04-20'").delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [29]:
import time
import random
pru = reload(pru)
nru = reload(nru)
import neuron
neuron = reload(neuron)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False,order="random")
print('Populate Done')

print(f"Total time for DecompositionSplit populate = {time.time() - start_time}")

Populate Started



---- Working on Neuron 864691136056340440 ----
Decompressing Neuron in minimal output mode...please wait
Skipping endnode 50 because skeletal distance was 2253.999109671855 and threshold was 2500


  0%|          | 0/3 [00:00<?, ?it/s]

***** there was no suggested cut for this limb even though it is still connnected***
n_paths_cut = 3
using precomputed split suggestions


---Working on Splitting Limb 0 with 7 components----


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1408070.  687160.  772202.]---------
Starting_edge inside branches_to_conept = [[1407296.66666667  679482.33333333  770757.66666667]
 [1408070.          687160.          772202.        ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [29]
printing out current edge:
[[1408070.          687160.          772202.        ]
 [1407296.66666667  679482.33333333  770757.66666667]]
edge_endpoints_to_process was empty so exiting loop after 40 iterations
starting_node in concept map (that should match the starting edge) = 32
Total time for branches to concept conversion = 1.0641064643859863

Done generating concept network 




----Working on seperate_graph 1-

Starting_edge inside branches_to_conept = [[1406973.25396825  679216.52380952  771475.31587302]
 [1406978.48846825  679221.75830952  771480.55037302]]
Done generating concept network 


Still remaining error limbs ([8.]), but will filter them away


------ Working on Soma 0 -------
limb_neighbors = [0, 1, 2, 3, 4, 5, 6, 8]
limb_neighbors BEFORE error limbs removed = [0, 1, 2, 3, 4, 5, 6, 8]
limb_neighbors AFTER error limbs removed = [0 1 2 3 4 5 6]
curr_n_multi_soma_limbs_cancelled = 0
curr_n_same_soma_limbs_cancelled = 1
n_errored_lims = 1
curr_error_limbs_cancelled_area = [1336.384686038517]
local_floating_meshes = [<trimesh.Trimesh(vertices.shape=(357, 3), faces.shape=(699, 3))>, <trimesh.Trimesh(vertices.shape=(97, 3), faces.shape=(190, 3))>, <trimesh.Trimesh(vertices.shape=(83, 3), faces.shape=(162, 3))>]
Already have preprocessed data
--- 1) Finished unpacking preprocessed materials: 0.00011205673217773438
total_edges = [['S0', 'L0'], ['S0', 'L1'], ['S0', 'L2'], ['S0', 'L3'], ['S

In [25]:
from pathlib import Path
curr_path = du.get_decomposition_path() / Path("864691135974454895_0_25.pbz2")
curr_path.exists()

False